###강아지 및 고양이 이진분류


###이미지 크롤링


In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.google.com/search?q=%EC%82%B0%EC%82%BC%EC%9E%8E&sca_esv=579077719&biw=1536&bih=715&gbv=1&tbm=isch&sxsrf=AM9HkKkoE8ujGwKJmrAs7mez_QtjJkoDwQ:1698995707203&ei=-51EZb6JDLK8hwP0gaXoDg&start=0&sa=N'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
img_tags = soup.select('img.DS1iW')
len(img_tags)

20

In [3]:
img_tags[0]

<img alt="" class="DS1iW" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2gJxM8hEm2Mb6WI09QlFfQPHpruHSEsMRAKbKFdcMxHGftXDGAu5OXFAXxw&amp;s"/>

In [4]:
img_tags[0]['src']

'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2gJxM8hEm2Mb6WI09QlFfQPHpruHSEsMRAKbKFdcMxHGftXDGAu5OXFAXxw&s'

In [5]:
import shutil
import os

# parent_folder에 folder_name이 기존에 없을 경우 새로 생성하는 함수 생성
def make_new_folder(folder_name, parent_folder):

    # Path
    path = os.path.join(parent_folder, folder_name)

    # Parent folder에 새로운 폴더 생성
    try:
        mode = 0o777

        # Path에 폴더 생성
        os.mkdir(path, mode)
    except OSError as error:
        print(error)

In [6]:
make_new_folder('ginseng leaf', '/content/')

In [7]:
path = '/content/ginseng leaf'

In [8]:
make_new_folder('산삼잎_2000장', '/content/')

In [9]:
path = '/content/산삼잎_2000장'

In [11]:
#산삼잎 2000개 저장
url = 'https://www.google.com/search?q=%EC%82%B0%EC%82%BC%EC%9E%8E&sca_esv=579077719&biw=1536&bih=715&gbv=1&tbm=isch&sxsrf=AM9HkKkoE8ujGwKJmrAs7mez_QtjJkoDwQ:1698995707203&ei=-51EZb6JDLK8hwP0gaXoDg&start={}&sa=N'

pages = [i for i in range(1001, 4001, 20)]

img_numbering = 0

for page in pages:
    url = url.format(page)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    img_tags = soup.select('img.DS1iW')

    for idx, img_tag in enumerate(img_tags):
        print( page//20 + 1,"페이지 ", img_tag['src'])
        img_resp = requests.get(img_tag['src'])

        # 산삼 파일에 저장ㅎ기
        img_path = os.path.join(path, f'산삼잎_{idx + 1 + img_numbering}.jpg')

        with open(img_path, 'wb') as f:
            f.write(img_resp.content)
    img_numbering += 20


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Google Cloud Console에서 생성한 프로젝트의 클라이언트 ID와 시크릿을 사용하여 로그인합니다.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

# GoogleDrive 객체 생성
drive = GoogleDrive(gauth)

# 나머지 코드는 이전과 동일하게 사용합니다.

In [ ]:


# 업로드할 로컬 폴더 경로
local_folder_path = '/content/산삼잎_2000장'  # 로컬 폴더 경로를 적절히 수정하세요.

# Google Drive에 업로드할 폴더 정보 생성
folder_drive = drive.CreateFile({'title': os.path.basename(local_folder_path), 'mimeType': 'application/vnd.google-apps.folder'})
folder_drive.Upload()

# 로컬 폴더 내 모든 파일 업로드
for file_name in os.listdir(local_folder_path):
    file_path = os.path.join(local_folder_path, file_name)
    file_drive = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_drive['id']}]})
    file_drive.Upload()

print("폴더와 파일이 성공적으로 업로드되었습니다.")